<a href="https://colab.research.google.com/github/Avradyooti/Practice/blob/main/SectionCDeepDive3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------- Load Data --------------------
daily_inventory = pd.read_csv('daily_inventory.csv', parse_dates=["snapshot_date"])
daily_sales = pd.read_csv('daily_sales.csv', parse_dates=["snapshot_date"])
purchase_orders = pd.read_csv('purchase_orders.csv')
products = pd.read_csv('products.csv')
web_events = pd.read_csv('web_events.csv', parse_dates=["event_ts"])

# -------------------- 1. Web Metrics: Sessions & Conversions --------------------
pageviews = web_events[web_events["event_type"] == "page_view"] \
    .groupby("product_id")["session_id"].nunique().rename("sessions")
purchases = web_events[web_events["event_type"] == "purchase"] \
    .groupby("product_id")["session_id"].nunique().rename("conversions")

product_metrics = pd.concat([pageviews, purchases], axis=1).fillna(0)
product_metrics["conversion_rate"] = product_metrics["conversions"] / product_metrics["sessions"]
product_metrics = product_metrics.reset_index()

# -------------------- 2. Add Style & Category Info --------------------
product_metrics = pd.merge(
    product_metrics,
    products[["product_id", "style_code", "category"]],
    on="product_id",
    how="left"
)

# -------------------- 3. Sell-through Calculation --------------------
sales_total = daily_sales.groupby("style_code")["units_sold"].sum().rename("units_sold_total")
latest_on_hand = daily_inventory.sort_values("snapshot_date").groupby("style_code").last()["on_hand_qty"].rename("on_hand_qty")

sell_through = (sales_total / (sales_total + latest_on_hand)).rename("sell_through")
sell_through_df = pd.concat([sales_total, latest_on_hand, sell_through], axis=1).reset_index()

# Merge with web metrics
product_metrics = pd.merge(product_metrics, sell_through_df, on="style_code", how="left")

# -------------------- 4. Identify Underperforming Category --------------------
low_perf_categories = product_metrics.groupby("category")["sell_through"].mean()
low_perf_categories = low_perf_categories[low_perf_categories < 0.40].sort_values()

if not low_perf_categories.empty:
    low_perf_category = low_perf_categories.index[0]
    category_df = product_metrics[product_metrics["category"] == low_perf_category]

    # -------------------- 5. Top 3 Styles by Sessions --------------------
    top3_styles = category_df.groupby("style_code")["sessions"].sum().sort_values(ascending=False).head(3).index.tolist()
    top3_data = category_df[category_df["style_code"].isin(top3_styles)][["style_code", "sessions", "conversion_rate"]].drop_duplicates()

    # -------------------- 6. Inventory vs Sales Plot --------------------
    inv_sales = pd.merge(
        daily_sales[daily_sales["style_code"].isin(top3_styles)],
        daily_inventory[daily_inventory["style_code"].isin(top3_styles)],
        on=["style_code", "snapshot_date"],
        how="inner"
    )

    plt.figure(figsize=(12, 6))
    for style in top3_styles:
        subset = inv_sales[inv_sales["style_code"] == style]
        plt.plot(subset["snapshot_date"], subset["on_hand_qty"], label=f"{style} - Inventory")
        plt.plot(subset["snapshot_date"], subset["units_sold"], linestyle='--', label=f"{style} - Sales")

    plt.title(f"Inventory vs Sales: Top 3 Styles in '{low_perf_category}'")
    plt.xlabel("Date")
    plt.ylabel("Units")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # -------------------- 7. Final Output --------------------
    print("Underperforming Category:", low_perf_category)
    print("Top 3 Styles by Sessions and Their Conversion Rates:")
    print(top3_data)

else:
    print("No category found with average sell-through below 40%. Consider using a higher threshold for testing.")

No category found with average sell-through below 40%. Consider using a higher threshold for testing.
